## Домашняя работа по теме "Рекурентные сети для обработки последовательностей"

In [1]:
import numpy as np
import pandas as pd

from string import punctuation
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head(3)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty


In [4]:
df['label'].value_counts() # распределение целевой переменной

0    29720
1     2242
Name: label, dtype: int64

In [5]:
# функция балансировки классов: 

def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1)

In [6]:
# балансировка классов целевой переменной: 

df_for_balancing = df
target_name = 'label'
df_balanced = balance_df_by_target(df_for_balancing, target_name)
    
df_balanced[target_name].value_counts()

C:\Users\Полина\AppData\Local\Temp\ipykernel_14132\838415103.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(sample, ignore_index=True)


0    29720
1    29146
Name: label, dtype: int64

In [7]:
# разобьем на тестовую и тренировачную выборку
df_train, df_val = train_test_split(df_balanced, test_size=0.3, random_state=42) #shuffle=True, stratify=df['label'], )

In [8]:
stop = set(get_stop_words("en"))

In [9]:
new_stop_words = {'im', 'u', 'us', '2', '2016', '4', '2017', 'ive', '3', '1', 'r', 'iam', 's', 'n', '10', 'ur', 'w', '8', '5', 'd', 'shes', '7', 'b', 'ag', 'bc', '6', 'x', '12', 'gt', 'whos', '1st', 'self', '50', 't', 'y'}

stop = stop.union(new_stop_words) 

In [10]:
puncts = set(punctuation)

In [11]:
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
#     txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in stop]
    return " ".join(txt)

In [12]:
df_train['tweet'].iloc[:1].values # до обработки

array(["every lactation consultant i've ever met has been a woman. hello? !"],
      dtype=object)

In [13]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values # после обработки

array(['every lactation consultant ever met woman hello'], dtype=object)

In [14]:
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

100%|██████████| 17660/17660 [00:04<00:00, 4230.11it/s]


In [15]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['every', 'lactation', 'consultant', 'ever', 'met']

In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [17]:
max_words = 3000
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

2999

In [18]:
tokens_filtered_top[:10]

['user',
 'amp',
 'love',
 'like',
 'trump',
 'just',
 'day',
 'will',
 'black',
 'people']

In [19]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [20]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [21]:
max_len = 30

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["tweet"]])

In [23]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [24]:
batch_size = 512

In [25]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [37]:
class GRUFixedLen(nn.Module) :

    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True) :
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)
        if self.use_last:
            last_tensor = gru_out[:,-1, :]
        else:
            last_tensor = torch.mean(gru_out[:,:], dim=1)
        out = self.linear(last_tensor)
        return torch.sigmoid(out)



    
gru_init = GRUFixedLen(max_words, 128, 64, use_last=False)
optimizer = torch.optim.Adam(gru_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [38]:
print(gru_init)
print(f'Parameters: {sum([param.nelement() for param in gru_init.parameters()])}')

GRUFixedLen(
  (embeddings): Embedding(3000, 128, padding_idx=0)
  (gru): GRU(128, 64, num_layers=2, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 446273


In [39]:
gru_init.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []

for epoch in range(epochs):  
    gru_init.train()
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = gru_init(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

    # выводим статистику о процессе обучения
    gru_init.eval()
    print(f'Epoch [{epoch+1}/{epochs}]. ' \
          f'Step [{i+1}/{len(train_loader)}]. ' \
          f'Loss: {loss:.3f}. ' \
          f'Acc: {running_right/running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1]
        test_outputs = gru_init(data[0])

        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))

        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    test_loss_history.append(test_loss)
    print(f'Test_loss: {test_loss:.3f}. Test_acc: {test_running_right/test_running_total:.3f}')


print('Training is finished!')

Epoch [1/5]. Step [81/81]. Loss: 0.326. Acc: 0.755. Test_loss: 0.213. Test_acc: 0.885
Epoch [2/5]. Step [81/81]. Loss: 0.187. Acc: 0.901. Test_loss: 0.120. Test_acc: 0.935
Epoch [3/5]. Step [81/81]. Loss: 0.144. Acc: 0.934. Test_loss: 0.043. Test_acc: 0.953
Epoch [4/5]. Step [81/81]. Loss: 0.141. Acc: 0.947. Test_loss: 0.048. Test_acc: 0.963
Epoch [5/5]. Step [81/81]. Loss: 0.075. Acc: 0.959. Test_loss: 0.034. Test_acc: 0.969
Training is finished!
